**<h1>TASK 1</h1>**

You are provided with data with 10-mins, 60-mins and 1-day resolution (Merge.csv)

Please merge them into a pandas Dataframe with 2-hours resolution in between 7:00 – 17:00 only as index. 

Please take the average of the 10-mins and 60-mins resolution prices during the 2-hours window and forward fill the 1-day resolution prices in the 2-hours window.

In [74]:
import pandas as pd
import numpy as np

datas=pd.read_csv("https://raw.githubusercontent.com/Meho96/test/main/Merge.csv")

In [75]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1570 entries, 0 to 1569
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Resolution  1570 non-null   object 
 1   Datetime    1570 non-null   object 
 2   Price       1488 non-null   float64
dtypes: float64(1), object(2)
memory usage: 36.9+ KB


In [76]:
datas.Resolution.unique()

array(['10MIN', '1H', 'D'], dtype=object)

In [77]:
#separate 10 minutes, 1 hour and Day resolution data into dataframes
ten_minutes_df=datas[datas["Resolution"]=="10MIN"].drop(["Resolution"],axis=1)
hour_df=datas[datas["Resolution"]=="1H"].drop(["Resolution"],axis=1)
day_df=datas[datas["Resolution"]=="D"].drop(["Resolution"],axis=1)

#fill Nan values
ten_minutes_df["Price"].fillna(0, inplace = True)
hour_df["Price"].fillna(0, inplace = True)
day_df["Price"].fillna(0, inplace = True)

#converting 10 minutes resolution data in 2 hours window by averaging prices
new_df=ten_minutes_df.groupby(ten_minutes_df.index // 12).agg({'Price': 'mean', 'Datetime': 'first'})
new_df=new_df.set_index("Datetime")

#converting 1 hours resolution data in 2 hours window by averaging prices
new_df_hour=hour_df.groupby(hour_df.index // 2).agg({'Price': 'mean', 'Datetime': 'first'})
new_df_hour=new_df_hour.set_index("Datetime")

day_df=day_df.set_index("Datetime")


In [78]:
#mergin dataframes by datetime
new_merge=pd.merge(pd.merge(new_df,new_df_hour, on="Datetime"),day_df,on="Datetime",how="outer")
new_merge.sort_index(inplace=True)
#forward filling 1day resolution prices
new_merge["Price"]=new_merge["Price"].ffill()
new_merge

,Price_x,Price_y,Price
Datetime,,,
2021-11-01 00:00:00,NaN,NaN,66.1
2021-11-01 07:00:00,59.047917,71.2625,66.1
2021-11-01 09:00:00,71.566667,71.0625,66.1
2021-11-01 11:00:00,68.812500,67.5750,66.1
2021-11-01 13:00:00,63.250000,63.7000,66.1
...,...,...,...
2021-11-30 07:00:00,56.779167,97.8750,91.7
2021-11-30 09:00:00,98.860417,98.6250,91.7
2021-11-30 11:00:00,96.662500,96.3625,91.7


**<h1>TASK 2</h1>**

You are provided with a daily energy consumption data from 2016 to date (Consumption.csv). 

Please create a Pandas DataFrame with to show the consumption of each year. 
The expected format is to have the year number as column name and mm-dd as index.

Please also create a seasonal plot showing 5-years (2016-2020) range (shaded) & average (dashed line), and year 2021 (line) & 2022 (line).

Please comment on your observation on the plot

In [79]:
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.graph_objects as go

consumption_data=pd.read_csv("https://raw.githubusercontent.com/Meho96/test/main/Consumption.csv")

In [80]:
consumption_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2468 entries, 0 to 2467
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         2468 non-null   object 
 1   Consumption  2468 non-null   float64
dtypes: float64(1), object(1)
memory usage: 38.7+ KB


In [81]:
#converting d/m/y (18/07/2016) and ymr (2016718) formats into one format
#dates like 2016111 can be either translated as 11/1/2016 or 1/11/2016. Which one to choose depends on the previous record of date in the dataset.
new_date=[]
for index,row in consumption_data.iterrows():
  sep = row.Date[2]
  if sep=='/':
    new_date.append(datetime.strptime(row["Date"],'%d/%m/%Y'))
  else:
    if len(row.Date)==8 or len(row.Date)==6:
      new_date.append(datetime.strptime(row["Date"],'%Y%m%d'))
    else:
      new_date.append(datetime.strptime(row["Date"],'%Y%m%d'))
      dd=new_date[index]-new_date[index-1]
      if dd.days>1:
        aa=datetime(int(row.Date[:4]), int(row.Date[4:5]), int(row.Date[-2:]))
        new_date = new_date[:index]+[aa]+new_date[index+1:]


consumption_data["newDate"]=new_date
consumption_data = consumption_data.drop('Date', axis=1)

In [82]:
consumption_data.head()

,Consumption,newDate
0,276.2910,2016-01-01
1,294.6780,2016-01-02
2,306.3096,2016-01-03
3,370.0566,2016-01-04
4,367.3998,2016-01-05


In [83]:
#new column for "year" and new column for "MM-DD" 
consumption_data['year'] = consumption_data['newDate'].dt.year
consumption_data['MM-DD'] = consumption_data['newDate'].dt.strftime('%m-%d')
consumption_data=consumption_data.drop("newDate", axis=1)
consumption_data.head()

,Consumption,year,MM-DD
0,276.2910,2016,01-01
1,294.6780,2016,01-02
2,306.3096,2016,01-03
3,370.0566,2016,01-04
4,367.3998,2016,01-05


In [84]:
# pivot table with "MM-DD" as index
table = pd.pivot_table(consumption_data,index='MM-DD',columns='year',values='Consumption', aggfunc=np.sum)  
print(table)

year       2016      2017      2018      2019      2020      2021      2022
MM-DD                                                                      
01-01  276.2910  339.5196  305.9370  313.6320  344.9952  344.0718  246.6612
01-02  294.6780  405.7614  365.8932  391.3758  380.8296  362.9772  263.8980
01-03  306.3096  407.3976  368.2746  424.1646  356.8698  365.1480  316.3698
01-04  370.0566  419.0454  372.1140  404.0766  348.2028  436.0716  348.6078
01-05  367.3998  457.1640  367.8534  364.0140  357.5502  446.6340  360.4338
...         ...       ...       ...       ...       ...       ...       ...
12-27  329.5728  347.8140  358.4736  337.5432  320.3550  330.2694       NaN
12-28  352.5606  357.4044  356.5296  343.2780  363.7872  311.3802       NaN
12-29  365.2614  351.6534  330.2370  338.0292  365.9094  299.7648       NaN
12-30  364.7754  307.8486  317.6334  343.9098  362.6694  269.2926       NaN
12-31  337.9482  278.9640  306.8442  336.7656  348.4620  245.5434       NaN

[366 rows x

In [85]:
#preparing data for seasonal plot. Range(Min-Max), Average, 2021, 2022
table['Max']=table.iloc[:,:4].max(axis=1)
table['Min']=table.iloc[:,:4].min(axis=1)
table['Average']=table.iloc[:,:4].mean(axis=1)
table['year2021']=table.iloc[:,5:6]
table['year2022']=table.iloc[:,6:7]

In [86]:
table

year,2016,2017,2018,2019,2020,2021,2022,Max,Min,Average,year2021,year2022
MM-DD,,,,,,,,,,,,
01-01,276.2910,339.5196,305.9370,313.6320,344.9952,344.0718,246.6612,339.5196,276.2910,308.84490,344.0718,246.6612
01-02,294.6780,405.7614,365.8932,391.3758,380.8296,362.9772,263.8980,405.7614,294.6780,364.42710,362.9772,263.8980
01-03,306.3096,407.3976,368.2746,424.1646,356.8698,365.1480,316.3698,424.1646,306.3096,376.53660,365.1480,316.3698
01-04,370.0566,419.0454,372.1140,404.0766,348.2028,436.0716,348.6078,419.0454,370.0566,391.32315,436.0716,348.6078
01-05,367.3998,457.1640,367.8534,364.0140,357.5502,446.6340,360.4338,457.1640,364.0140,389.10780,446.6340,360.4338
...,...,...,...,...,...,...,...,...,...,...,...,...
12-27,329.5728,347.8140,358.4736,337.5432,320.3550,330.2694,NaN,358.4736,329.5728,343.35090,330.2694,NaN
12-28,352.5606,357.4044,356.5296,343.2780,363.7872,311.3802,NaN,357.4044,343.2780,352.44315,311.3802,NaN
12-29,365.2614,351.6534,330.2370,338.0292,365.9094,299.7648,NaN,365.2614,330.2370,346.29525,299.7648,NaN


In [87]:
#seasonal plot
p1_fig = go.Figure()


# Set up lists to iterate through
fig_list = [p1_fig]
pivot_list = [table]
title_list = ['Seasonal plot 2016-2020 range and average, 2021, 2022']

# Set the colors for the lines
colors = ['#17becf','#e377c2','#ff7f0e','#2ca02c','darkblue','firebrick']

# Create x-axis labels for the months
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

# Create the Graphs
for j in range(1):
    data = pivot_list[j]

    fig_list[j].add_trace(go.Scatter(x=months, y=data['Max'],
        fill=None,
        mode=None,
        line_color='lightgray', showlegend=True,name="Max"
        ))

    fig_list[j].add_trace(go.Scatter(x=months, y=data['Min'],
        fill='tonexty', # fill area between trace0 and trace1
        mode=None, line_color='lightgray', showlegend=True,name="Min"))

    fig_list[j].add_trace(go.Scatter(x=months, y=data['Average'], name='Average',
                                 line=dict(color='black', width=4, dash='dot')))
    
    fig_list[j].add_trace(go.Scatter(x=months, y=data['year2021'], name='2021',
                                 line=dict(color='red', width=4)))
    
    fig_list[j].add_trace(go.Scatter(x=months, y=data['year2022'], name='2022',
                                 line=dict(color='#ff7f0e', width=4)))
       
    #if you wanna show plot for each year individually, set different value in range
    for i in range(0):
        fig_list[j].add_trace(go.Scatter(x=months, y=data.iloc[:,i], name=data.columns[i],
                                 line=dict(color=colors[i], width=4)))

    fig_list[j].update_layout(title=title_list[j],
                       xaxis_title='Month',
                       yaxis_title='Energy consumption', template='plotly_white')
    
# Show all the graphs
p1_fig.show()



**<h1>TASK 3</h1>**

A robot moves around a flat surface from position (0,0). It accepts instructions, and moves accordingly.

For example:

“BEGIN”

LEFT 3

UP 5

RIGHT 4

DOWN 7

The first word indicates direction and the number shows steps.
The robot will stop moving with instruction “STOP”.

Please write a function, which accepts instructions as a list. When first “STOP” instruction is given, it calculates the distance of Robot from the original position (0,0)

In [88]:
import math
import re

#list of instructions
instructionList=['Begin',"Right 10","Up 4", "Left 8","STOP","Down 4"]

#function that seperates instruction into direction and steps
def splitInstruction(s):
    direction = s.rstrip('0123456789')
    steps = s[len(direction):]
    return direction, steps


def calculateDistance(instructionList):
  position=[0,0]
  direction=[splitInstruction(s)[0] for s in instructionList]
  direction = [i.upper() for i in direction]
  steps=[splitInstruction(s)[1] for s in instructionList]
  i=0

  while i<len(direction):
    if 'BEGIN' in direction[i]: position[0] == 0
    elif 'UP' in direction[i]: position[0] += int(steps[i])
    elif 'DOWN' in direction[i]: position[0] -= int(steps[i])
    elif 'RIGHT' in direction[i]: position[1] += int(steps[i])
    elif 'LEFT' in direction[i]: position[1] -= int(steps[i])
    elif 'STOP' in direction[i]: break
    i+=1  
  #Euclidian distance  
  print(float(math.sqrt(position[1]**2+position[0]**2)))
   

calculateDistance(instructionList)  


4.47213595499958


**<h1>TASK 4</h1>**

You are provided with sample trades data in a data file (trades.csv).

It contains a collection of trade data during 5 working days with volume and price. You are asked to process the sample data into a data set for creating the candlestick plots, with open, high, low & close prices (OHLC) and total volume for each product and contract, over a time interval.

You are expected to write a function to create a dataframe contains OHLC and trading volume with(begin, end, product(s), freq) as input. It should be able take freq input, e.g., 15MIN, 1H, 1D.

Expected Result – A python model file detailing process with comments:

Note:

• The product “Emission - Venue A” and “Emission - Venue B” are the same product trading in two different venues, please combine them when queried

• If product list has more than one contract, please generate OHLC and volume data for each contract

• Please limit output within trading hours, i.e., 7:00 – 17:00, except when freq >= 1D

• Please do not use OHLC resampler

In [89]:
import pandas as pd
from datetime import datetime
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots


trades_data=pd.read_csv("https://raw.githubusercontent.com/Meho96/test/main/Trades.csv")

In [90]:
trades_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70095 entries, 0 to 70094
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TradeDateTime  70095 non-null  object 
 1   Product        70095 non-null  object 
 2   Contract       70095 non-null  object 
 3   Price          70095 non-null  float64
 4   Quantity       70095 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 2.7+ MB


In [91]:
#convert object to datetime format
trades_data["TradeDateTime"] = pd.to_datetime(trades_data["TradeDateTime"], format='%d/%m/%Y %H:%M')

In [92]:
def plotData(contractData,freq):
  contractsData=contractData
  contractsData=contractsData.set_index("TradeDateTime")
  if freq !="1D":
    contractsData=contractsData.between_time("09:00","17:00")

  resample_Price = contractsData['Price'].resample(freq).ohlc(_method='ohlc')
  resample_Volume = contractsData['Quantity'].resample(freq).sum()
  contractsData = pd.concat([resample_Price, resample_Volume], axis=1,)

  # Create subplots and mention plot grid size
  fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.03, subplot_titles=('OHLC', 'Volume'), 
               row_width=[0.2, 0.7])
  
  # Plot OHLC on 1st row
  fig.add_trace(go.Candlestick(x=contractsData.index, open=contractsData["open"], high=contractsData["high"],
                low=contractsData["low"], close=contractsData["close"], name="OHLC"), 
                row=1, col=1)
  
  # Bar trace for volumes on 2nd row without legend
  fig.add_trace(go.Bar(x=contractsData.index, y=contractsData['Quantity'], showlegend=False), row=2, col=1)

  # Do not show OHLC's rangeslider plot 
  fig.update(layout_xaxis_rangeslider_visible=False)
  fig.show()



def createDataframe(begin,end,products,freq,tradesdata):
  new_dataframe=tradesdata;
  new_dataframe=new_dataframe[(new_dataframe['TradeDateTime'] >= begin) & (new_dataframe['TradeDateTime'] <= end)]

  if (products=="Emission - Venue A") or (products=="Emission - Venue B"):
    new_dataframe=new_dataframe[(new_dataframe["Product"]=="Emission - Venue A") | (new_dataframe["Product"]=="Emission - Venue B")]
    nContracts=new_dataframe.Contract.unique()
    for x in nContracts:
      contractDataframe=new_dataframe[(new_dataframe["Contract"]==str(x))]
      plotData(contractDataframe,freq)
  else:
    new_dataframe=new_dataframe[(new_dataframe["Product"]==products)]
    nContracts=new_dataframe.Contract.unique()
    for x in nContracts:
      contractDataframe=new_dataframe[(new_dataframe["Contract"]==str(x))]
      plotData(contractDataframe,freq)


In [93]:
trades_data.Product.unique()

array(['Emission - Venue A', 'Energy', 'Emission - Venue B'], dtype=object)

In [95]:
createDataframe("2022-04-18 00:37:00","2022-04-20 00:37:00","Emission - Venue A","15Min",trades_data)

In [ ]:
createDataframe("2022-04-18 00:37:00","2022-04-26 00:37:00","Energy","15Min",trades_data)